# Part I. ETL Pipeline for Pre-Processing the Files

#### Import Python packages 

In [1]:
# Import Python packages 
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv

#### Creating list of filepaths to process original event csv data files

In [2]:
print(os.getcwd())

filepath = os.getcwd() + '/event_data'

for root, dirs, files in os.walk(filepath):
    file_path_list = glob.glob(os.path.join(root,'*'))

/home/workspace


#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [3]:
full_data_rows_list = [] 
    

for f in file_path_list:
     with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        # creating a csv reader object 
        csvreader = csv.reader(csvfile) 
        next(csvreader)       
        for line in csvreader:
            full_data_rows_list.append(line) 
            
print(len(full_data_rows_list))

csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))


8056


In [34]:
# number of rows after cut off the useless rows for this project
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

6821


# Part II. Complete the Apache Cassandra coding portion of your project. 

## Now you are ready to work with the CSV file titled <font color=red>event_datafile_new.csv</font>, located within the Workspace directory.  The event_datafile_new.csv contains the following columns: 
- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId

The image below is a screenshot of what the denormalized data should appear like in the <font color=red>**event_datafile_new.csv**</font> after the code above is run:<br>

<img src="images/image_event_datafile_new.jpg">

## Create the Apache Cassandra Database

#### Creating a Cluster

In [5]:
from cassandra.cluster import Cluster
cluster = Cluster(['127.0.0.1'])
session = cluster.connect()

#### Create Keyspace

In [6]:
try:
    session.execute("""
                CREATE KEYSPACE IF NOT EXISTS udacity
                WITH REPLICATION =
                {'class' : 'SimpleStrategy' , 'replication_factor' : 1}
                """)
except Exception as e:
        print(e)

#### Set Keyspace

In [7]:
try:
    session.set_keyspace('udacity')
except Exception as e:
        print(e)

## Create queries to ask the following three questions of the data

### 1. Give me the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4

In [8]:
# Create the table for the first Query (Question)
query = "CREATE TABLE IF NOT EXISTS session_table"
query = query + "(session_id int, item_ID int, artist text, song_title text, duration double, PRIMARY KEY (session_id, item_ID))"
try:
    session.execute(query)
except Exception as e:
    print(e)

In [9]:
# read the file and insert the values into the table
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = "INSERT INTO session_table (session_id, item_id, artist, song_title, duration)"
        query = query + "VALUES (%s, %s, %s, %s, %s)"
        session.execute(query, (int(line[8]), int(line[3]), line[0], line[9], float(line[5])))

In [33]:
# Execute the first query and constructing a pandas DataFrame
query = "SELECT * FROM session_table WHERE session_Id=338 AND item_id=4"
df = pd.DataFrame(list(session.execute(query)))

In [32]:
# Print the result of the query
df.head()

,session_id,item_id,artist,duration,song_title
0,338,4,Faithless,495.3073,Music Matters (Mark Knight Dub)


### 2. Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182

In [17]:
# create the table for the second Query
query = "CREATE TABLE IF NOT EXISTS user_table"
query = query + "(user_id int, session_ID int, artist text,itemInSession int, song_title text, first_name text, last_name text, PRIMARY KEY ((user_id, session_ID), itemInSession))"
try:
    session.execute(query)
except Exception as e:
    print(e)

In [19]:
# read the file and insert the values into the table
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = "INSERT INTO user_table (user_id, session_id, itemInSession, artist, song_title, first_name, last_name)"
        query = query + "VALUES (%s, %s, %s, %s, %s, %s, %s)"
        session.execute(query, (int(line[10]), int(line[8]),int(line[3]), line[0], line[9], line[1], line[4]))

                    

In [31]:
# Execute the second query and constructing a pandas DataFrame
query = "SELECT artist, song_title, first_name, last_name FROM user_table WHERE user_id = 10 AND session_id = 182"
df1 = pd.DataFrame(list(session.execute(query)))

In [23]:
# Print the result of the query
df1.head()

,artist,song_title,first_name,last_name
0,Down To The Bone,Keep On Keepin' On,Sylvie,Cruz
1,Three Drives,Greece 2000,Sylvie,Cruz
2,Sebastien Tellier,Kilometer,Sylvie,Cruz
3,Lonnie Gordon,Catch You Baby (Steve Pitron & Max Sanna Radio...,Sylvie,Cruz


### 3. Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'

In [25]:
# create the table for the third Query
query = "CREATE TABLE IF NOT EXISTS listen_songs"
query = query + "(user_id int, song_title text, first_name text, last_name text, PRIMARY KEY (song_title, user_id))"
try:
    session.execute(query)
except Exception as e:
    print(e)

In [27]:
# read the file and insert the values into the table
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:

        query = "INSERT INTO listen_songs (song_title, user_id, first_name, last_name)"
        query = query + "VALUES (%s, %s, %s, %s)"
        session.execute(query, (line[9], int(line[10]), line[1], line[4]))

                    

In [29]:
# Execute the third query and constructing pandas DataFrame
query = "SELECT first_name, last_name FROM listen_songs WHERE song_title = 'All Hands Against His Own'"
df2 = pd.DataFrame(list(session.execute(query)))

In [30]:
# Print the result of the query
df2.head()

,first_name,last_name
0,Jacqueline,Lynch
1,Tegan,Levine
2,Sara,Johnson


### Drop the tables

In [23]:
query = "DROP TABLE session_table"
try:
    session.execute(query)
except Exception as e:
    print(e)

query = "DROP TABLE user_table"
try:
    session.execute(query)
except Exception as e:
    print(e)

query = "DROP TABLE listen_songs"
try:
    session.execute(query)
except Exception as e:
    print(e)

In [24]:
# shutdown the session and close the connection
session.shutdown()
cluster.shutdown()